# Sort missed, detected sparse cells

author: steeve.laquitaine@epfl.ch


* `534` near-contact cells
* `472` pyramidal cells


setup virtual env env_kilosort_silico

In [7]:
# SETUP PACKAGES 
%load_ext autoreload
%autoreload 2
import os 
import pandas as pd

# SET PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.pipes.sorting import match_sorted_to_true_neuropixels_2023_02_19 as match
from src.nodes.postpro import feateng

# SET PARAMETERS
EXPERIMENT = "silico_neuropixels"   # specifies the experiment 
SIMULATION_DATE = "2023_02_19"      # specifies the run (date)


# SETUP CONFIG
data_conf, param_conf = get_config(EXPERIMENT, SIMULATION_DATE).values()


# SETUP PATH
CELL_MATCHING_PATH = data_conf["postprocessing"]["cell_matching"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-05-26 14:59:58,085 - root - utils.py - get_config - INFO - Reading experiment config.
2023-05-26 14:59:58,113 - root - utils.py - get_config - INFO - Reading experiment config. - done


## Sort missed, detected sparse firing cells

In [3]:
# stats: 40 secs, 534 cells, 1 sorter
# create Matching object 
out = match.run()

# load created Matching object
cell_matching = pd.read_parquet(CELL_MATCHING_PATH)

# label missed and detected cells
cell_matching = feateng.add_is_missed(cell_matching, data_conf)

2023-05-26 14:58:51,949 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - loading kilosort3 Sorting Extractor
2023-05-26 14:58:51,964 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - loading kilosort3 Sorting Extractor - done: 0.0
2023-05-26 14:58:51,965 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - loading Ground truth Sorting Extractor
2023-05-26 14:58:51,985 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - loading Ground truth Sorting Extractor - done: 0.0
2023-05-26 14:59:08,925 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - Detecting true cell oversplit - done
2023-05-26 14:59:08,933 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - Detecting true cell misses - done
2023-05-26 14:59:08,992 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - loading kilosort3 Sorting Extractor
2023-05-26 14:59:09,006 - root - match_sorted_to_true_

/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/src/pipes/sorting/match_sorted_to_true_neuropixels_2023_02_19.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_matching["oversplit_true_cell"].iloc[oversplit.index] = True


2023-05-26 14:59:19,548 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - Detecting true cell oversplit - done
2023-05-26 14:59:19,553 - root - match_sorted_to_true_neuropixels_2023_02_19.py - run - INFO - Detecting true cell misses - done


/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spike-sorting/src/pipes/sorting/match_sorted_to_true_neuropixels_2023_02_19.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cell_matching["oversplit_true_cell"].iloc[oversplit.index] = True


In [4]:
# find missed and detected
df = cell_matching.drop_duplicates(subset=['true_cell_match'])
detected_sparse_cells = df[(df["true firing rate"]<=0.2) & (df["detection_label"]=="detected")]["true_cell_match"]
missed_sparse_cells = df[(df["true firing rate"]<=0.2) & (df["detection_label"]=="missed")]["true_cell_match"]

In [11]:
any(missed_sparse_cells==165135)

True

TODO: Why does no sorted cell match 165135, while I detect spikes for 165135? Is cell_matching wrong?

In [14]:
df[df["true_cell_match"]==165135]

,sorted_cell,true_cell_match,oversplit_true_cell,agreement_score,true firing rate,ks3 firing rate,detection_label
517,NaN,165135,None,NaN,0.120002,NaN,missed
